In [1]:
import sys
sys.path.append('..')

In [2]:
import warnings
warnings.filterwarnings("ignore")

import pickle
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.patches as mpatches

from scipy.stats import entropy
from scipy.stats import pearsonr
from matplotlib import pyplot as plt
from sklearn.feature_selection import f_regression

from trickster.wfp_helper import load_data, extract

In [3]:
plt.rc('text', usetex=True)
plt.rc('font', family='sans-serif')
plt.rc('pdf', fonttype=42)
plt.rc('ps', fonttype=42)

matplotlib.rcParams['text.latex.preamble'] = [
#       r'\usepackage{helvet}',
       r'\usepackage{siunitx}',
       r'\sisetup{detect-all}',
       r'\usepackage{sansmath}',  
       r'\sansmath'             
]

sns.set_context('paper', font_scale=2.5)

In [16]:
epsilons = [3, 4, 5, 10, 15, 25, 30, 40]
max_iter = 7000

In [4]:
results_pickle_names = ['../out/results_full_eps_{}.pkl'.format(eps) for eps in epsilons]
adv_results = [pickle.load(open(filename, 'rb')) for filename in results_pickle_names]

## Runtime and success rates

In [5]:
expanded_df = pd.DataFrame(columns=['eps', 'nodes'])

for eps, results_df in zip(epsilons, adv_results):
    for i, row in results_df.iterrows():
        if row.found:
            expanded_df = expanded_df.append(
                {'eps': eps, 'nodes': float(row.nodes_expanded)},
                ignore_index=True)

In [6]:
success_rates = [results_df.found.mean() for results_df in adv_results]

Text(0,0.5,'Attack success')

In [34]:
fig, ax1 = plt.subplots()

sns.lineplot(x='eps', y='nodes', data=expanded_df, ax=ax1)
ax1.hlines(max_iter, min(epsilons), 4)
ax1.set_xlim(min(epsilons), max(epsilons))
ax1.set_yscale('log')
ax1.set_ylabel('Nodes expanded')
ax1.set_xlabel('$\\varepsilon$')

ax2 = ax1.twinx()
ax2.plot(epsilons, success_rates, color='orange')
ax2.set_ylim(0., 1 + .05)
ax2.set_ylabel('Attack success')

## Cost overhead

In [9]:
costs_df = pd.DataFrame(columns=['eps', 'index', 'cost', 'overhead'])

baseline_df = adv_results[0]

for eps, results_df in zip(epsilons, adv_results):
    for i, row in results_df.iterrows():
        if row.found:
            try:
                _, baseline_item = next(
                    baseline_df[baseline_df['index']==row['index']].iterrows())
            except:
                continue

            if baseline_item.found:
                overhead = row.path_cost / baseline_item['real_cost']
                costs_df = costs_df.append(
                    {'eps': eps, 'index': row['index'], 'cost': row['path_cost'], 'overhead': overhead},
                    ignore_index=True)

In [38]:
fig, ax = plt.subplots()

sns.lineplot(x='eps', y='overhead', data=costs_df,
             legend=False, ax=ax)
# ax.plot(epsilons, np.array(epsilons) / epsilons[0] * 2, label='Worst-case overhead', linestyle='--')
ax.set_xlim(min(epsilons) - 0.1, max(epsilons))
# ax.set_ylim(1, 2.5)
ax.set_ylabel('Cost overhead')
ax1.set_xlabel('$\\varepsilon$')
ax